## Capstone project notebook: Clustering Toronto neighborhoods (week #3)

By William Gruintal 

This notebook will be used for the capstone project assignment for week #3

In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
#retrienving the URL and creating a Beautiful soup object
URL = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(URL).text
soup = BeautifulSoup(html_data, 'html5lib')
#print(soup.prettify())

In [3]:
#extract data from table
table_contents = []
table = soup.find('table')
#for row in table.findAll('td'):
for row in table.find_all('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace('/', ',')).replace(')', ' ')).strip(' ')
        table_contents.append(cell)

In [39]:
#print (table_contents)
df = pd.DataFrame(table_contents)
df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                       'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                       'EtobicokeNorthwest':'Etobicoke Northwest',
                       'East YorkEast Toronto':'East York/East Toronto',
                       'MississaugaCanada Post Gateway Processing Centre':'Mississauga'}, inplace = True)
#df.head(12)
df.shape

#print(df.Borough.unique())
#df.head()

(103, 3)

In [35]:
df['Neighborhood'].isnull().sum()

0

In [36]:
lat_lng_coords = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
print(lat_lng_coords.shape)
lat_lng_coords.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [37]:
lat_lng_coords.rename(columns = {'Postal Code': 'PostalCode'}, inplace = True)

print(df.columns)

print(lat_lng_coords.columns)

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')
Index(['PostalCode', 'Latitude', 'Longitude'], dtype='object')


In [38]:
toronto_neighborhoods = pd.merge(left = df, right = lat_lng_coords, left_on = 'PostalCode', right_on = 'PostalCode')
toronto_neighborhoods

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,...",43.636258,-79.498509
